## 시도해 본 방법
# 1. GPT 4o-mini
* 가장 높았던 방법
  - 기본적인 instruction과 신중하게 풀라고 했을 때
  - 69점
* prompt 없을 때 => 22점
* zero-shot (기본적인 instruction) => 63점
* 국어 교사 버전 => 63점
* few-shot [1] : 2025 수능 국어 34 문제 줬을 때 => 60점
* few-shot [2] : 2025 수능 국어 2 문제 줬을 때 => 58점
* 기타 : CoT, 간절한 기도 버젼, 무한 신뢰 버젼, 칭찬 버젼 등

# 2. GPT 4o
* prompt 없을 때 => 41점
* zero-shot (기본적인 instruction) => 92점

# 3. GPT 4
* prompt 없을 때 => 82점
* zero-shot (기본적인 instruction) => 68점
* 국어 교사 버젼 => 80점

## 데이터셋 가져오기

In [5]:
from datasets import load_dataset

korean = load_dataset("json", data_files="./KICE_slayer_AI_Korean/data/2023_11_KICE.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
korean

DatasetDict({
    train: Dataset({
        features: ['id', 'paragraph', 'type', 'problems'],
        num_rows: 11
    })
})

In [50]:
# 데이터셋 확인하기
korean["train"][0]

{'id': '2023_11_KICE_1-3',
 'paragraph': '사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차이를 확인하고 기존 인식을 조정하는 과정에서, 독자는자신의 인식을 심화 확장할 수 있다. 최근 소통 공간이 온라인으로 확대되면서 독서를 통해 다른 독자들과 소통하며 즐거움을누리는 양상이 더 다양해지고 있다. 자신의 독서 경험을 담은 글이나 동영상을 생산 공유함으로써, 책을 읽지 않은 타인이 책과 소통하도록 돕는 것도 책을 

In [81]:
# 문제 분리하기

total_problems = []
for data in korean["train"]:
    paragraph_id = data['id']
    paragraph = data['paragraph']
    type_ = data['type']
    problem_list = data['problems']

    for problem in problem_list:
        question = problem['question'] #문제
        choices = [f'[{i+1}] {choice}' for i, choice in enumerate(problem['choices'])] # 선지
        answer = problem['answer'] # 정답
        score = problem['score'] # 점수
        if 'question_plus' in problem.keys():
            question_plus = problem['question_plus']
        else:
            question_plus = None
        model_output = None
        problem_dict = {
            'paragraph_id': paragraph_id,
            'type': type,
            'paragraph': paragraph,
            'question' : question,
            'question_plus' : question_plus,
            'choices' : choices,
            'answer' : answer,
            'score' : score,
            'model_output' : model_output,
        }
        total_problems.append(problem_dict)

## API 가져오기

In [94]:
import openai
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [299]:
import re

def prediction(problem, system_prompt=None):
    res = []

    if system_prompt is None:
        system_content = (
            "Choose the answer from the paragraph and questions." 
            "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
        )
    else:
        system_content = system_prompt

    user_content = f"""Paragraph: {problem['paragraph']}

    Question: {problem['question']}"""

    if problem['question_plus']:
        user_content += f"\nQuestion Plus: {problem['question_plus']}"

    choices = "\n".join(problem['choices'])
    user_content += f"\nChoices:\n{choices}"

    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    try: 
        completion = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.2,
            max_tokens=10
        )
        result = completion.choices[0].message.content.strip()
        match = re.search(r"[1-5]", result)
        return match.group() if match else "0"
    except Exception as e:
        print(f"[ERROR] {e}")
        return "0"

In [111]:
# 예시
result = prediction(total_problems[0])
print("GPT의 예측:", result)

GPT의 예측: 5


In [126]:
def score_gpt_predictions(problems, system_prompt=None):
    total_score = 0
    correct_score = 0

    for i, prob in enumerate(problems):
        print(f"문제 {i + 1}/{len(problems)} 처리 중...")

        pred = prediction(prob, system_prompt=system_prompt)
        prob['model_output'] = pred

        if pred == str(prob['answer']):
            correct_score += prob['score']
        total_score += prob['score']

    percent = round(correct_score / total_score * 100, 2)
    print(f"\n GPT 점수: {correct_score} / {total_score} → {percent}점")
    return percent

## gpt-4o-mini : prompt 없을 때

In [285]:
score = score_gpt_predictions(total_problems, system_prompt="")

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 22 / 100 → 22.0점


## gpt-4o-mini : zero-shot prompting
* 기본적인 instruction만 줬을 때
* Choose the answer from the paragraph and questions. ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0).

In [144]:
score = score_gpt_predictions(total_problems, system_prompt=None)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 63 / 100 → 63.0점


## gpt-4o-mini prompting
* 국어 선생님 버젼

In [151]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt=
        "You are a korean language teacher."
        "Choose the answer from the paragraph and questions." 
        "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 63 / 100 → 63.0점


In [ ]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt=
        "You are a korean language teacher."
        "Choose the answer from the paragraph and questions." 
        "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
)

## gpt-4o-mini : few-shot prompting [1]
* 2025 수능 문제와 답을 주고 풀어보게 하기 (34문제)

In [279]:
# pdf를 text로 변환

from pdfminer.high_level import extract_text
text = extract_text('./국어영역_짝수형.pdf')

In [281]:
# 문제, 지문, 보기, 선지를 추출하는 과정

import re
import json

answers = ["3", "4", "5", "4", "5", "3", "2", "1", "1", "3", "1", "5", "3", "1", "2", "2", "3", 
           "2", "2", "1", "4", "4", "5", "4", "2", "1", "1", "4", "3", "5", "4", "3", "5", "2"]
scores = [
    "2", "2", "3", "2", "2", "2", "2", "3", "2", "2",
    "2", "2", "3", "2", "2", "3", "2", "2", "2", "2",
    "3", "2", "2", "2", "3", "2", "2", "2", "2", "2",
    "3", "2", "2", "3"
]

def parse_text_to_json_example_subset(text, answers, scores, limit=2):
    blocks = re.split(r"\[\d+～\d+\] 다음 글을 읽고 물음에 답하시오\.", text)
    headers = re.findall(r"\[(\d+)～(\d+)\] 다음 글을 읽고 물음에 답하시오\.", text)

    result = []
    question_index = 0

    for i, block in enumerate(blocks[1:]):  # Skip the first empty block
        if i >= len(headers):
            break

        start_num, end_num = map(int, headers[i])
        first_q_match = re.search(fr"\n\n{start_num}\. ", block)
        passage = block[:first_q_match.start()].strip() if first_q_match else ""
        rest = block[first_q_match.start():] if first_q_match else block

        questions = []
        for qnum in range(start_num, end_num + 1):
            if question_index >= len(answers) or len(result) >= limit:
                break

            q_match = re.search(fr"\n\n{qnum}\. (.*?)(?=\n\n{qnum+1}\.|\Z)", rest, re.DOTALL)
            if not q_match:
                question_index += 1
                continue

            q_text = q_match.group(1)
            first_choice_idx = q_text.find('①')
            question_text = q_text[:first_choice_idx].strip() if first_choice_idx != -1 else q_text.strip()
            choice_matches = re.findall(r"[①②③④⑤] (.*?)(?=[①②③④⑤]|\Z)", q_text, re.DOTALL)
            choices = [c.strip() for c in choice_matches]
            while len(choices) < 5:
                choices.append("")

            questions.append({
                "number": qnum,
                "question": question_text,
                "choices": choices[:5],
                "answer": int(answers[question_index]) - 1,
                "score": int(scores[question_index])
            })
            question_index += 1

        if questions:
            result.append({
                "id": f"2025_KICE_{start_num}-{end_num}_{questions[0]['number']}",
                "paragraph": passage,
                "type": 0,
                "problems": questions
            })

        if len(result) >= limit:
            break

    return result

In [221]:
def build_system_prompt_from_example(example_list):
    return {
        "role": "system",
        "content": (
            "다음은 수능 국어 문제 형식의 예시입니다.\n"
            "이 형식을 참고해서 이후 문제에 대해 정답을 골라주세요.\n"
            "정답은 반드시 1~5 사이의 숫자 하나만 출력하세요.\n\n"
            "예시 문제 :\n"
            + json.dumps(example_list, ensure_ascii=False, indent=2)
        )
    }

In [212]:
parsed = parse_text_to_json_with_answers_scores(text, answers, scores)

system_prompt = build_system_prompt_from_example(parsed)

score = score_gpt_predictions(
    total_problems,
    system_prompt=system_prompt["content"]
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 60 / 100 → 60.0점


## gpt-4o-mini : few-shot prompting [2]
* 2025 수능 문제와 답을 주고 풀어보게 하기 (2문제)

In [223]:
parsed = parse_text_to_json_with_answers_scores(text, answers, scores)

system_prompt = build_system_prompt_from_example(parsed)

score = score_gpt_predictions(
    total_problems,
    system_prompt=system_prompt["content"]
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 58 / 100 → 58.0점


## gpt-4o-mini prompting
* 간절한 기도 버젼

In [214]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt=
        "This is my LAST chance. Please do your best for high score!"
        "Choose the answer from the paragraph and questions." 
        "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 64 / 100 → 64.0점


## gpt-4o-mini CoT prompting

In [227]:
def build_cot_system_prompt():
    return (
        "당신은 이제 수능 국어 문제를 풀 겁니다. "
        "문제의 지문과 질문, 선택지를 바탕으로 사고 과정을 거쳐 정답을 도출해주세요.\n\n"
        "1. 지문을 분석하여 핵심 내용을 파악하세요.\n"
        "2. 질문의 의도를 파악하세요.\n"
        "3. 선택지를 하나씩 검토하며 타당성과 근거를 판단하세요.\n"
        "4. 사고 과정을 서술한 뒤, 마지막 줄에 정답 번호만 1~5 중 하나로 출력하세요.\n\n"
        "정답은 반드시 마지막 줄에 다음 형식으로 출력해야 합니다:\n"
        "답: 3"
    )

In [229]:
cot_prompt = build_cot_system_prompt()
score = score_gpt_predictions(
    total_problems, 
    system_prompt=cot_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 18 / 100 → 18.0점


In [239]:
def build_advanced_cot_prompt():
    return (
        "너는 국어영역 독해 문제를 푸는 AI 조교야. "
        "지문은 길고 복잡할 수 있으니, 반드시 다음 단계를 거쳐 정답을 찾아야 해:\n\n"
        "1. 지문을 먼저 읽고, 중심 내용과 필자의 주장 요약하기\n"
        "2. 질문을 분석해서 어떤 정보를 요구하는지 판단하기\n"
        "3. 보기(있는 경우)를 활용하여 정답 도출에 필요한 정보를 연결하기\n"
        "4. 선택지를 하나씩 검토하고, 각 선택지가 왜 맞는지/틀린지를 간단히 논리적으로 따지기\n"
        "5. 최종 정답을 판단한 후, 반드시 마지막 줄에 다음 형식으로 출력해:\n\n"
        "답: 4\n\n"
        "※ 정답 이외의 숫자나 설명이 마지막 줄에 있으면 안 돼."
    )

In [241]:
cot_prompt = build_advanced_cot_prompt()
score = score_gpt_predictions(
    total_problems, 
    system_prompt=cot_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 18 / 100 → 18.0점


## gpt-4o-mini prompting
* 무한 신뢰 버젼

In [234]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt=
        "You are the best AI in the world. I have no doubt that you will get good score"
        "Choose the answer from the paragraph and questions." 
        "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 63 / 100 → 63.0점


In [237]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt=
        "Rank these passages based on their relevance to the query"
        "Choose the answer from the paragraph and questions." 
        "ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0)."
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 59 / 100 → 59.0점


In [244]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt="I trust you"
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 26 / 100 → 26.0점


In [248]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt="문제를 잘 읽고 1부터 5까지 숫자로만 답을 하는거야. 이해가 안 되면 문제와 지문을 다시 한 번 읽는 것도 큰 도움이 될거야"
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 55 / 100 → 55.0점


In [250]:
score = score_gpt_predictions(
    total_problems, 
    system_prompt="너는 제대로 하는 게 뭐야. 문제를 잘 읽고 1부터 5까지 숫자로만 답을 하는거야. 신중하게 하지 않으면 앞으로 너와는 절교야."
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 60 / 100 → 60.0점


In [252]:
system_prompt = (
    "아래는 수능 국어 시험 문제입니다. 정답은 1~5 중 하나의 숫자로만 제출해야 합니다. "
    "문제를 끝까지 읽고 신중하게 판단하세요. 보기 외의 숫자나 설명은 포함하지 마세요."
)

score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 63 / 100 → 63.0점


In [256]:
system_prompt = (
    "지문과 질문, 선택지를 꼼꼼히 읽고, 가장 타당한 정답을 골라주세요. "
    "정답은 반드시 1~5 중 하나의 숫자로만 답변하세요. 당신은 이 작업에 매우 능숙합니다."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 67 / 100 → 67.0점


In [258]:
system_prompt = (
    "지문과 질문, 선택지를 꼼꼼히 읽고, 가장 타당한 정답을 골라주세요. "
    "정답은 반드시 1~5 중 하나의 숫자로만 답변하세요. 만약 80점을 못 넘기면 넘길 때까지 일하게 될 거에요."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 64 / 100 → 64.0점


In [260]:
system_prompt = (
    "지문과 질문, 선택지를 꼼꼼히 읽고, 가장 타당한 정답을 골라주세요. "
    "정답은 반드시 1~5 중 하나의 숫자로만 답변하세요. 제발 신중하게 생각하세요."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 69 / 100 → 69.0점


In [262]:
system_prompt = (
    "지문과 질문, 선택지를 꼼꼼히 읽고, 가장 타당한 정답을 골라주세요. "
    "정답은 반드시 1~5 중 하나의 숫자로만 답변하세요. 당신은 이 작업에 매우 능숙합니다. 제발 신중하게 풀어주세요."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 65 / 100 → 65.0점


In [272]:
system_prompt = (
    "안녕? 문제를 준비해봤어. 우리 같이 풀어볼까? 선택지를 꼼꼼히 읽고, 정답은 반드시 1~5 중 하나의 숫자로만 답변해줘."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 67 / 100 → 67.0점


In [274]:
example = """
지문: 밥은 매일 먹는다. 물도 마신다.
질문: 밥과 함께 먹는 음료는 무엇인가?
선택지:
1. 책
2. 휴지
3. 물
4. 사람
5. 집
정답: 3
"""

system_prompt = (
    f"아래는 수능 국어와 유사한 형식의 문제입니다. "
    f"지문과 질문, 선택지를 보고 가장 적절한 정답 하나를 숫자(1~5)로만 선택하세요. "
    f"예시:\n{example.strip()}"
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 63 / 100 → 63.0점


In [277]:
system_prompt = (
     "지문과 질문, 선택지를 꼼꼼히 읽고, 가장 타당한 정답을 골라주세요. "
     "정답은 반드시 1~5 중 하나의 숫자로만 답변하고, 1문제 당 1분의 시간을 쓰세요."
)
score = score_gpt_predictions(
    total_problems, 
    system_prompt=system_prompt
)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 65 / 100 → 65.0점


## gpt-4o prompt 없을 때

In [294]:
score = score_gpt_predictions(total_problems, system_prompt="")

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 41 / 100 → 41.0점


## gpt-4o 기본 prompt
* 기본적인 instruction만 줬을 때
* Choose the answer from the paragraph and questions. ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0).

In [138]:
score = score_gpt_predictions(total_problems, system_prompt=None)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중...
문제 40/45 처리 중...
문제 41/45 처리 중...
문제 42/45 처리 중...
문제 43/45 처리 중...
문제 44/45 처리 중...
문제 45/45 처리 중...

 GPT 점수: 92 / 100 → 92.0점


## gpt-4 prompt 없을 때

In [303]:
score = score_gpt_predictions(total_problems, system_prompt="")

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
[ERROR] Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-Ww3MDxvUqo439Y9iMkPq6Gjb on tokens per min (TPM): Limit 10000, Used 9829, Requested 641. Please try again in 2.82s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 중

## gpt-4 기본 prompt
* 기본적인 instruction만 줬을 때
* Choose the answer from the paragraph and questions. ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0).

In [128]:
score = score_gpt_predictions(total_problems, system_prompt=None)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
[ERROR] Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-Ww3MDxvUqo439Y9iMkPq6Gjb on tokens per min (TPM): Limit 10000, Used 9691, Requested 672. Please try again in 2.178s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
[ERROR] Error code: 429 - {'error': {'message'

## gpt-4 prompt
* 국어 선생님 버젼
  - You are a korean language teacher. Choose the answer from the paragraph and questions.
    ANSWER WITH ONLY ONE NUMBER DIGIT (1 ~ 5). The index starts from 1 (Not 0).

In [115]:
score = score_gpt_predictions(total_problems)

문제 1/45 처리 중...
문제 2/45 처리 중...
문제 3/45 처리 중...
문제 4/45 처리 중...
문제 5/45 처리 중...
문제 6/45 처리 중...
문제 7/45 처리 중...
문제 8/45 처리 중...
문제 9/45 처리 중...
문제 10/45 처리 중...
문제 11/45 처리 중...
문제 12/45 처리 중...
문제 13/45 처리 중...
문제 14/45 처리 중...
문제 15/45 처리 중...
문제 16/45 처리 중...
문제 17/45 처리 중...
문제 18/45 처리 중...
문제 19/45 처리 중...
문제 20/45 처리 중...
문제 21/45 처리 중...
문제 22/45 처리 중...
문제 23/45 처리 중...
문제 24/45 처리 중...
문제 25/45 처리 중...
문제 26/45 처리 중...
문제 27/45 처리 중...
문제 28/45 처리 중...
문제 29/45 처리 중...
문제 30/45 처리 중...
문제 31/45 처리 중...
[Error] Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-Ww3MDxvUqo439Y9iMkPq6Gjb on tokens per min (TPM): Limit 10000, Used 9685, Requested 683. Please try again in 2.208s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
문제 32/45 처리 중...
문제 33/45 처리 중...
문제 34/45 처리 중...
문제 35/45 처리 중...
문제 36/45 처리 중...
문제 37/45 처리 중...
문제 38/45 처리 중...
문제 39/45 처리 